In [4]:
import imageio
import io
from django.template.loader import render_to_string
from anymail.message import AnymailMessage
import logging
from django.core.files.images import ImageFile

from print_nanny_webapp.client_events.tasks import predict_events_dataframe
from print_nanny_webapp.client_events.models import PredictEvent, AlertMessage

df = predict_events_dataframe(4)

def send_email_notification(df, ratio=0.55):
    frame_ids = [frame_id for frame_id, _ in df.index]

    predict_events = PredictEvent.objects.filter(
        id__in=frame_ids,
        print_job=4        
    ).order_by('dt').all()

    print_job = predict_events[0].print_job

    last_alert_message = AlertMessage.objects.filter(
        print_job=print_job.id,
    ).order_by('-created_dt').first()
    if last_alert_message is not None:
        if last_alert_message.last_action != AlertMessage.ActionChoices.RESUME_ALERTS:
            logging.warning('Alert for print job {print_job} already sent at {alert_message.created_dt}, skipping')
            return

    images = []

    for i, event in enumerate(predict_events):
        images.append(imageio.imread(event.files.annotated_image.file))
        print(f'finished {i}/{len(predict_events)}')
    buff = io.BytesIO()
    imageio.mimwrite(buff, images, fps=20, format='GIF-PIL')
    buff.seek(0)
    
    filename= f'print_job_{print_job.id}_alert_message_{predict_events[0].dt}_{predict_events[len(predict_events)-1].dt}'
    img_file = ImageFile(buff, name=filename)
    alert_message = AlertMessage.objects.create(
        user=print_job.user,
        print_job=print_job,
        video=img_file,
        dataframe=df.to_dict(orient='split')
    )
    
    
    merge_data = {
        'RATIO': '{:.2%}'.format(ratio),
        'GCODE_FILE': print_job.gcode_file.name,
        'VIDEO_URL': alert_message.video.url,
        'STOP_URL': f'http://localhost/feedback/{alert_message.id}?action=stop_print',
        'RESUME_URL': f'http://localhost/feedback/{alert_message.id}?action=resume',
        'FIRST_NAME': print_job.user.first_name or 'Maker',
    }

    subject = render_to_string("email/print_alert_message_subject.txt", merge_data).strip()
    text_body = render_to_string("email/print_alert_message_body.txt", merge_data)
    html_body = render_to_string("email/print_alert_message_body.html", merge_data)

    message = AnymailMessage(
        subject=subject,
        body=text_body,
        to=[predict_events[0].user.email],
        tags=["default-print-alert"],  # Anymail extra in constructor
    )
    message.attach_alternative(html_body, 'text/html')
    return message.send()

send_email_notification(df)


INFO 2020-11-19 15:45:15,321 tasks 165 139935208851264 Analyzing 41 predict_events for print_job 4


finished 0/41
finished 1/41
finished 2/41
finished 3/41
finished 4/41
finished 5/41
finished 6/41
finished 7/41
finished 8/41
finished 9/41
finished 10/41
finished 11/41
finished 12/41
finished 13/41
finished 14/41
finished 15/41
finished 16/41
finished 17/41
finished 18/41
finished 19/41
finished 20/41
finished 21/41
finished 22/41
finished 23/41
finished 24/41
finished 25/41
finished 26/41
finished 27/41
finished 28/41
finished 29/41
finished 30/41
finished 31/41
finished 32/41
finished 33/41
finished 34/41
finished 35/41
finished 36/41
finished 37/41
finished 38/41
finished 39/41
finished 40/41


1